In [1]:
import numpy as np
import pypsa

In [2]:
# marginal costs in EUR/MWh
marginal_cost = {"Wind": 0, "Hydro": 0, "Coal": 30, "Gas": 60, "Oil": 80}

# power plant capacities (nominal powers in MW) in each country (not necessarily realistic)
power_plant_p_nom_ = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {
        "Hydro": 1200,
    },
    "Swaziland": {
        "Hydro": 600,
    },
}

# transmission capacities in MW (not necessarily realistic)
transmission = {
    "South Africa": {"Mozambique": 500, "Swaziland": 250},
    "Mozambique": {"Swaziland": 100},
}

# country electrical loads in MW (not necessarily realistic)
loads = {"South Africa": 42000, "Mozambique": 650, "Swaziland": 250}

In [3]:
network=pypsa.Network()
countries = ["Mozambique", "South Africa"]

for country in countries:
    network.add("Bus", country)

    for fuel_type in power_plant_p_nom_[country]:
        network.add(
            "Generator",
            f"{country} {fuel_type}",
            bus=country,
            p_nom=power_plant_p_nom_[country][fuel_type],
            marginal_cost=marginal_cost[fuel_type],
        )

    network.add("Load", f"{country} load", bus=country, p_set=loads[country])

    # add transmission as controllable Link
    if country not in transmission:
        continue

    for other_country in countries:
        if other_country not in transmission[country]:
            continue

        # NB: Link is by default unidirectional, so have to set p_min_pu = -1
        # to allow bidirectional (i.e. also negative) flow
        network.add(
            "Link",
            f"{country} - {other_country} link",
            bus0=country,
            bus1=other_country,
            p_nom=transmission[country][other_country],
            p_min_pu=-1,
        )

In [4]:
network.optimize()

Index(['South Africa - Mozambique link'], dtype='object', name='Link')
Index(['Mozambique', 'South Africa'], dtype='object', name='Bus')
Index(['South Africa - Mozambique link'], dtype='object', name='Link')
Index(['Mozambique', 'South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 14 duals
Objective: 1.26e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


('ok', 'optimal')

In [5]:
network.loads_t.p

Load,Mozambique load,South Africa load
snapshot,,
now,650.0,42000.0


In [7]:
network.generators_t.p

Generator,Mozambique Hydro,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,,
now,1150.0,35000.0,3000.0,3500.0,-0.0


In [11]:
network.links_t.p1

Link,South Africa - Mozambique link
snapshot,
now,500.0


In [12]:
network.buses_t.marginal_price

Bus,Mozambique,South Africa
snapshot,,
now,-0.0,60.0


In [13]:
network = pypsa.Network()

countries = ["Swaziland", "Mozambique", "South Africa"]

for country in countries:
    network.add("Bus", country)

    for fuel_type in power_plant_p_nom_[country]:
        network.add(
            "Generator",
            f"{country} {fuel_type}",
            bus=country,
            p_nom=power_plant_p_nom_[country][fuel_type],
            marginal_cost=marginal_cost[fuel_type],
        )

    network.add("Load", f"{country} load", bus=country, p_set=loads[country])

    # add transmission as controllable Link
    if country not in transmission:
        continue

    for other_country in countries:
        if other_country not in transmission[country]:
            continue

        # NB: Link is by default unidirectional, so have to set p_min_pu = -1
        # to allow bidirectional (i.e. also negative) flow
        network.add(
            "Link",
            f"{country} - {other_country} link",
            bus0=country,
            bus1=other_country,
            p_nom=transmission[country][other_country],
            p_min_pu=-1,
        )

In [15]:
network.optimize()

Index(['Mozambique - Swaziland link', 'South Africa - Swaziland link',
       'South Africa - Mozambique link'],
      dtype='object', name='Link')
Index(['Swaziland', 'Mozambique', 'South Africa'], dtype='object', name='Bus')
Index(['Mozambique - Swaziland link', 'South Africa - Swaziland link',
       'South Africa - Mozambique link'],
      dtype='object', name='Link')
Index(['Swaziland', 'Mozambique', 'South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.3s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 9 primals, 21 duals
Objective: 1.24e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


('ok', 'optimal')

In [16]:
network.loads_t.p

Load,Swaziland load,Mozambique load,South Africa load
snapshot,,,
now,250.0,650.0,42000.0
